In [ ]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4 MB 17.4 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051301 sha256=5ca3b35bf4103556dc6613a9f91a3af7df3cff25cb2819b6dd92a9680bbe6f5b
  Stored in directory: /tmp/pip-ephem-wheel-cache-mlw5fiba/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


Now Restart Runtime

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q torch torchvision

In [ ]:
pip install transformers

     |████████████████████████████████| 3.8 MB 21.6 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
     |████████████████████████████████| 6.5 MB 23.5 MB/s 
     |████████████████████████████████| 67 kB 2.2 MB/s 
     |████████████████████████████████| 895 kB 46.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
!pip install datasets transformers

     |████████████████████████████████| 311 kB 17.4 MB/s 
     |████████████████████████████████| 134 kB 52.8 MB/s 
     |████████████████████████████████| 212 kB 74.6 MB/s 
     |████████████████████████████████| 1.1 MB 46.1 MB/s 
     |████████████████████████████████| 271 kB 54.6 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 57.8 MB/s 


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.17.0


# Fine-tuning a model on a multiple choice task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a multiple choice task, which is the task of selecting the most plausible inputs in a given selection. The dataset used here is [SWAG](https://www.aclweb.org/anthology/D18-1009/) but you can adapt the pre-processing to any other multiple choice dataset you like, or your own data. SWAG is a dataset about commonsense reasoning, where each example describes a situation then proposes four options that could go after it. 

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a mutiple choice head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those two parameters, then the rest of the notebook should run smoothly:

In [ ]:
model_checkpoint = "bert-base-uncased"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data. This can be easily done with the functions `load_dataset`.  

In [ ]:
from datasets import load_dataset, load_metric

`load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [ ]:
# datasets_swag = load_dataset("swag", "regular")

In [ ]:
datasets = load_dataset("super_glue", "copa")

Downloading:   0%|          | 0.00/9.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset super_glue downloaded and prepared to /root/.cache/huggingface/datasets/super_glue/copa/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['premise', 'choice1', 'choice2', 'question', 'idx', 'label'],
        num_rows: 400
    })
    validation: Dataset({
        features: ['premise', 'choice1', 'choice2', 'question', 'idx', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['premise', 'choice1', 'choice2', 'question', 'idx', 'label'],
        num_rows: 500
    })
})

To access an actual element, you need to select a split first, then give an index:

In [ ]:
datasets["train"][0]

{'choice1': 'The sun was rising.',
 'choice2': 'The grass was cut.',
 'idx': 0,
 'label': 0,
 'premise': 'My body cast a shadow over the grass.',
 'question': 'cause'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,premise,choice1,choice2,question,idx,label
0,The girl made a wish.,She saw a black cat.,She saw a shooting star.,cause,45,choice2
1,The bully stuck his foot out in front of his classmate.,The bully picked up his classmate.,The bully's classmate tripped.,effect,274,choice2
2,The woman was wrongly convicted for the crime.,The jury was fair.,She was framed.,cause,229,choice2
3,The woman presented her brother with a gift.,Her brother refused to accept the gift.,Her brother unwrapped the gift.,effect,317,choice2
4,The engine of the airplane was faulty.,The airplane crashed.,The pilot made an error.,effect,254,choice1
5,I wrote a thank-you note to my grandmother.,She became forgetful.,She sent me a gift.,cause,138,choice2
6,The man took notice of the woman.,He caught a whiff of her perfume.,He had his back turned to her.,cause,272,choice1
7,The football coach lost his voice after the game.,He yelled whenever a player fumbled the ball.,He whistled whenever a player scored a touchdown.,cause,308,choice1
8,The woman spoke with a foreign accent.,She came from a wealthy family.,She migrated from another country.,cause,81,choice2
9,The patient underwent the risky medical procedure.,The procedure was costly.,Specialists recommended the procedure.,cause,360,choice2


Each example in the dataset has a context composed of a first sentence (in the field `sent1`) and an introduction to the second sentence (in the field `sent2`). Then four possible endings are given (in the fields `ending0`, `ending1`, `ending2` and `ending3`) and the model must pick the right one (indicated in the field `label`). The following function lets us visualize a give example a bit better:

In [ ]:
def show_one(example):
    print(f"Premise: {example['premise']}")
    print(f"Question: {example['question']}")
    print(f"  Choice1 - {example['choice1']}")
    print(f"  Choice2 - {example['choice2']}")
    print(f"\nLabel: {['Choice1', 'Choice2'][example['label']]}")

In [ ]:
show_one(datasets["train"][0])

Premise: My body cast a shadow over the grass.
Question: cause
  Choice1 - The sun was rising.
  Choice2 - The grass was cut.

Label: Choice1


In [ ]:
show_one(datasets["train"][15])

Premise: The girl gasped.
Question: cause
  Choice1 - Her friend stuck an ice cube down her back.
  Choice2 - Her friend gave her a pat on the back.

Label: Choice1


## Knowledge Base

In [ ]:
import pickle

with open("/content/drive/MyDrive/NLP/graph", "rb") as input_file:
  G = pickle.load(input_file)

G.number_of_edges()

5829960

### Finding Paths
Choosing the Path based on Edge Scores¶

In [ ]:
def find_max_score_path(G, source, target, cutoff):
    paths = list(nx.all_simple_paths(G, source=source, target=target, cutoff=cutoff))
    scores = []
    for path in paths:
        pathGraph = nx.path_graph(path)
        pathScore = 1
        for e in pathGraph.edges():
            pathScore *= G.edges[e[0], e[1]]['weight']
        scores.append(pathScore)
    return paths[np.argmax(scores)], np.max(scores)

def find_path(source_word, target_word):
    s_path = nx.shortest_path(G, source=source_word, target=target_word)
    pathGraph = nx.path_graph(s_path)
    pathScore = 0
    for e in pathGraph.edges():
        pathScore += G.edges[e[0], e[1]]['weight']
    return s_path, pathScore/len(s_path)-1

### Path to text

In [ ]:
mapping_templates = {
    "RelatedTo": "{0} is related to {1}.",
    "ExternalURL": "{0} is described at the following URL {1}.",
    "FormOf": "{0} is a form of the word {1}.",
    "IsA": "{0} is a {1}.",
    "PartOf": "{1} has {0}.",
    "HasA": "{0} has {1}.",
    "UsedFor": "{0} is used for {1}.",
    "CapableOf": "{0} can {1}.",
    "AtLocation": "You are likely to find {0} in {1}.",
    "Causes": "Sometimes {0} causes {1}.",
    "HasSubevent": "Something you might do while {0} is {1}.",
    "HasFirstSubevent": "the first thing you do when you {0} is {1}.",
    "HasLastSubevent": "the last thing you do when you {0} is {1}.",
    "HasPrerequisite": "something you need to do before you {0} is {1}.",
    "HasProperty": "{0} is {1}.",
    "HasContext": "{0} is a word used in the context of {1}.",
    "MotivatedByGoal": "You would {0} because you want to {1}.",
    "ObstructedBy": "{0} can be prevented by {1}.",
    "Desires": "{0} wants {1}.",
    "CreatedBy": "{0} is created by {1}.",
    "Synonym": "{0} and {1} are have similar meanings.",
    "Antonym": "{0} is the opposite of {1}.",
    "DistinctFrom": "it cannot be both {0} and {1}.",
    "DerivedFrom": "the word {0} is derived from the word {1}.",
    "SymbolOf": "{0} is a symbol of {1}.",
    "DefinedAs": "{0} is defined as {1}.",
    "Entails": "if {0} is happening, {1} is also happening.",
    "MannerOf": "{0} is a specific way of doing {1}.",
    "LocatedNear": "{0} is located near {1}.",
    "dbpedia": "{0} is conceptually related to {1}.",
    "SimilarTo": "{0} is similar to {1}.",
    "EtymologicallyRelatedTo": "the word {0} and the word {1} have the same origin.",
    "EtymologicallyDerivedFrom": "the word {0} comes from the word {1}.",
    "CausesDesire": "{0} makes people want {1}.",
    "MadeOf": "{0} is made of {1}.",
    "ReceivesAction": "{0} can be {1}.",
    "InstanceOf": "{0} is an example of {1}.",
    "NotDesires": "{0} does not want {1}.",
    "NotUsedFor": "{0} is not used for {1}.",
    "NotCapableOf": "{0} is not capable of {1}.",
    "NotHasProperty": "{0} does not have the property of {1}.",
    "NotMadeOf": "{0} is not made of {1}.",
    "NotIsA": "{0} is not a {1}.",
    
    "RelatedTo -1": "{1} is related to {0}.",
    "ExternalURL -1": "{1} is described at the following URL {0}.",
    "FormOf -1": "{1} is a form of the word {0}.",
    "IsA -1": "{1} is a {0}.",
    "PartOf -1": "{0} has {1}.",
    "HasA -1": "{1} has {0}.",
    "UsedFor -1": "{1} is used for {0}.",
    "CapableOf -1": "{1} can {0}.",
    "AtLocation -1": "You are likely to find {1} in {0}.",
    "Causes -1": "Sometimes {1} causes {0}.",
    "HasSubevent -1": "Something you might do while {1} is {0}.",
    "HasFirstSubevent -1": "the first thing you do when you {1} is {0}.",
    "HasLastSubevent -1": "the last thing you do when you {1} is {0}.",
    "HasPrerequisite -1": "something you need to do before you {1} is {0}.",
    "HasProperty -1": "{1} is {0}.",
    "HasContext -1": "{1} is a word used in the context of {0}.",
    "MotivatedByGoal -1": "You would {1} because you want to {0}.",
    "ObstructedBy -1": "{1} can be prevented by {0}.",
    "Desires -1": "{1} wants {0}.",
    "CreatedBy -1": "{1} is created by {0}.",
    "Synonym -1": "{1} and {0} are have similar meanings.",
    "Antonym -1": "{1} is the opposite of {0}.",
    "DistinctFrom -1": "it cannot be both {1} and {0}.",
    "DerivedFrom -1": "the word {1} is derived from the word {0}.",
    "SymbolOf -1": "{1} is a symbol of {0}.",
    "DefinedAs -1": "{1} is defined as {0}.",
    "Entails -1": "if {1} is happening, {0} is also happening.",
    "MannerOf -1": "{1} is a specific way of doing {0}.",
    "LocatedNear -1": "{1} is located near {0}.",
    "dbpedia -1": "{1} is conceptually related to {0}.",
    "SimilarTo -1": "{1} is similar to {0}.",
    "EtymologicallyRelatedTo -1": "the word {1} and the word {0} have the same origin.",
    "EtymologicallyDerivedFrom -1": "the word {1} comes from the word {0}.",
    "CausesDesire -1": "{1} makes people want {0}.",
    "MadeOf -1": "{1} is made of {0}.",
    "ReceivesAction -1": "{1} can be {0}.",
    "InstanceOf -1": "{1} is an example of {0}.",
    "NotDesires -1": "{1} does not want {0}.",
    "NotUsedFor -1": "{1} is not used for {0}.",
    "NotCapableOf -1": "{1} is not capable of {0}.",
    "NotHasProperty -1": "{1} does not have the property of {0}.",
    "NotMadeOf -1": "{1} is not made of {0}.",
    "NotIsA -1": "{1} is not a {0}."
}

In [ ]:
import networkx as nx

def path_to_text(G, path):
  path_text = ""
  pathGraph = nx.path_graph(path)
  for e in pathGraph.edges():
    rel = G.edges[e[0], e[1]]['relation']
    sentence = mapping_templates[rel].format(e[0], e[1])
    path_text += sentence + " "
  return path_text

def path_analysis(G, path):
  path_text = ""
  edge_types = []

  pathGraph = nx.path_graph(path)
  for e in pathGraph.edges():
    # print(e, G.edges[e[0], e[1]])
    rel = G.edges[e[0], e[1]]['relation']
    edge_types.append(rel)
    sentence = mapping_templates[rel].format(e[0], e[1])
    path_text += sentence + " "

  return len(path)-1, edge_types, path_text

### Extracting Terms

In [ ]:
import spacy
from string import punctuation
import numpy as np

nlp = spacy.load("en_core_web_md")


def extract_keywords(text):
    result = []
    doc = nlp(text.lower())
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        result.append(token.text)
    return result

### Find Q&A paths

We sort all of the paths for the question and answer using the path score and return the top 3 paths

In [ ]:
def convert_sent_to_text(sentences):
  text = ""
  for sent in sentences:
    text += sent
  
  return text

def find_QandA_path(G, premise, choice):
    paths = []

    # print('Q&A paths for: ', premise, choice)
    for p_word in premise:
      for word in choice:
        if (not(p_word in G) or (not(word in G))):
          continue
        if word != p_word:
          s_path, s_path_score = find_path(p_word, word)
          length, edges, path_text = path_analysis(G, s_path)
          path_text = path_to_text(G, s_path)
          paths.append((path_text, s_path_score))
    
    most_relevant_paths = max(paths,key=lambda item:item[1])[0]
    return most_relevant_paths

def find_QandA_paths(G, premise, choice):
    paths = []

    # print('Q&A paths for: ', premise, choice)
    for p_word in premise:
      for word in choice:
        if (not(p_word in G) or (not(word in G))):
          continue
        if word != p_word:
          s_path, s_path_score = find_path(p_word, word)
          length, edges, path_text = path_analysis(G, s_path)
          path_text = path_to_text(G, s_path)
          paths.append((path_text, s_path_score))
    if len(paths) == 0:
      return ""
    paths.sort(key=lambda y: y[1], reverse=True)
    max_index = np.min([3, len(paths)])
    most_relevant_paths = [elem[0] for elem in paths[:max_index]]
    return convert_sent_to_text(most_relevant_paths)    

In [ ]:
a = extract_keywords("The only baggage the woman checked was a drawstring bag.")
b = extract_keywords("garbage can")

find_QandA_paths(G, a, b)


'checked is related to check. check is a word used in the context of us. garbage is a word used in the context of us. curb_service is related to baggage. curb_service is related to garbage. the word garbagewoman is derived from the word woman. garbagewoman is related to garbage. '

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

We can then write the function that will preprocess our samples. The tricky part is to put all the possible pairs of sentences in two big lists before passing them to the tokenizer, then un-flatten the result so that each example has four input ids, attentions masks, etc.

When calling the `tokenizer`, we use the argument `truncation=True`. This will ensure that an input longer than what the model selected can handle will be truncated to the maximum length accepted by the model.

In [ ]:
choice_names = ["choice1", "choice2"]

def preprocess_function(examples):
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    first_sentences = [[context] * 2 for context in examples["premise"]]
    # Grab all second sentences possible for each context.
    question_headers = []
    for question in examples["question"]:
      if(question == "cause"):
        question_headers.append("Because, ")
      else:
       question_headers.append("So, ")
    second_sentences = [[f"{header} {examples[choice][i]}" for choice in choice_names] for i, header in enumerate(question_headers)]
    
    third_sentences = [[find_QandA_paths(G, extract_keywords(premise), extract_keywords(examples[choice][i])) for choice in choice_names] for i, premise in enumerate(examples["premise"])]

    context = []
    for i in range(len(examples["premise"])):
      context_row = []
      for j in range(len(choice_names)):
        sentence = first_sentences[i][j] + third_sentences[i][j]
        context_row.append(sentence)
      context.append(context_row)

    # Flatten everything
    context = sum(context, [])
    second_sentences = sum(second_sentences, [])
    
    # Tokenize
    tokenized_examples = tokenizer(context, second_sentences, truncation=True)
    # Un-flatten
    return {k: [v[i:i+2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists of lists for each key: a list of all examples (here 5), then a list of all choices (4) and a list of input IDs (length varying here since we did not apply any padding):

In [ ]:
examples = datasets["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 2 [63, 60]


To check we didn't do anything group when grouping all possibilites then unflattening, let's have a look at the decoded inputs for a given example:

In [ ]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(2)]

['[CLS] the runner wore shorts. you are likely to find shorts in sporting _ goods _ store. you are likely to find puck in sporting _ goods _ store. the word where _ puck _ is _ heading is derived from the word puck. where _ puck _ is _ heading is related to predicted. wore is a word used in the context of computing. element is a word used in the context of computing. ekamanganese is related to element. ekamanganese is related to predicted. wore is a word used in the context of computing. volatile is a word used in the context of computing. cryocondensation is related to volatile. cryocondensation is related to temperatures. [SEP] because, the forecast predicted high temperatures. [SEP]',
 '[CLS] the runner wore shorts. runner is a device. device is a design. design is related to planned. you are likely to find runner in track. track is a bar. bar is related to beach. you are likely to find shorts in soccer _ game. you are likely to find ball in soccer _ game. ball is related to beach. 

We can compare it to the ground truth:

In [ ]:
show_one(datasets["train"][3])

Premise: The runner wore shorts.
Question: cause
  Choice1 - The forecast predicted high temperatures.
  Choice2 - She planned to run along the beach.

Label: Choice1


This seems alright, so we can apply this function on all the examples in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our task is about mutliple choice, we use the `AutoModelForMultipleChoice` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-copa",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    logging_steps=5
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-swag"` or `"huggingface/bert-finetuned-swag"`).

Then we need to tell our `Trainer` how to form batches from the pre-processed inputs. We haven't done any padding yet because we will pad each batch to the maximum length inside the batch (instead of doing so with the maximum length of the whole dataset). This will be the job of the *data collator*. A data collator takes a list of examples and converts them to a batch (by, in our case, applying padding). Since there is no data collator in the library that works on our specific problem, we will write one, adapted from the `DataCollatorWithPadding`:

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

When called on a list of examples, it will flatten all the inputs/attentions masks etc. in big lists that it will pass to the `tokenizer.pad` method. This will return a dictionary with big tensors (of shape `(batch_size * 4) x seq_length`) that we then unflatten.

We can check this data collator works on a list of features, we just have to make sure to remove all features that are not inputs accepted by our model (something the `Trainer` will do automatically for us after):

In [ ]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

Again, all those flatten/un-flatten are sources of potential errors so let's make another sanity check on our inputs:

In [ ]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(2)]

['[CLS] the physician misdiagnosed the patient. patient is a case. case and lawsuit are have similar meanings. misdiagnosed is a form of the word misdiagnose. misdiagnose is related to diagnose. diagnose is a word used in the context of medicine. case is a word used in the context of medicine. case and lawsuit are have similar meanings. misdiagnosed is a form of the word misdiagnose. misdiagnose is related to diagnose. diagnose is a word used in the context of medicine. medicine is related to practice. the word malpractice is derived from the word practice. [SEP] so, the patient filed a malpractice lawsuit against the physician. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] the physician misdiagnosed the patient. patient is the opposite of agent. agent is related to secret. undisclosed is related to secret. undisclosed is related to disclosed. misdiagnosed is a form of the word misdiagnose. misdiagnose is related to diagnose. diagnose is a word used in the context of medici

In [ ]:
show_one(datasets["train"][8])

Premise: The physician misdiagnosed the patient.
Question: effect
  Choice1 - The patient filed a malpractice lawsuit against the physician.
  Choice2 - The patient disclosed confidential information to the physician.

Label: Choice1


All good!

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits:

In [ ]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/Ssadaf/bert-base-uncased-finetuned-copa into local empty directory.


We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice2, idx, choice1, premise, question. If choice2, idx, choice1, premise, question are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 400
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 75


Epoch,Training Loss,Validation Loss,Accuracy
1,0.703800,0.686324,0.530000
2,0.590200,0.685398,0.660000
3,0.434000,0.719998,0.670000


The following columns in the evaluation set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice2, idx, choice1, premise, question. If choice2, idx, choice1, premise, question are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice2, idx, choice1, premise, question. If choice2, idx, choice1, premise, question are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice2, idx, choice1, premise, question. If choice2, idx, choice1, premise, question are n

TrainOutput(global_step=75, training_loss=0.6114299329121907, metrics={'train_runtime': 53.3065, 'train_samples_per_second': 22.511, 'train_steps_per_second': 1.407, 'total_flos': 158292767421312.0, 'train_loss': 0.6114299329121907, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(encoded_datasets["validation"])

The following columns in the test set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice2, idx, choice1, premise, question. If choice2, idx, choice1, premise, question are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 100
  Batch size = 16


In [ ]:
predicted_labels = []
for pred in predictions[0]:
  if(pred[0] > pred[1]):
    predicted_labels.append(0)
  else:
    predicted_labels.append(1)

In [ ]:
predictions[2]

{'test_accuracy': 0.6700000166893005,
 'test_loss': 0.7199977040290833,
 'test_runtime': 1.3275,
 'test_samples_per_second': 75.328,
 'test_steps_per_second': 5.273}

In [ ]:
val_labels = encoded_datasets["validation"]["label"]

In [ ]:
from itertools import compress

choosen_indexes = [not(predicted_labels[i] == val_labels[i]) for i in range(len(val_labels))]
mistakes = list(compress(datasets["validation"], choosen_indexes))
len(mistakes)

33

In [ ]:
for i in range(25):
  show_one(mistakes[i])
  print("_________________________\n")

Premise: I wanted to conserve energy.
Question: effect
  Choice1 - I swept the floor in the unoccupied room.
  Choice2 - I shut off the light in the unoccupied room.

Label: Choice2
_________________________

Premise: The hamburger meat browned.
Question: cause
  Choice1 - The cook froze it.
  Choice2 - The cook grilled it.

Label: Choice2
_________________________

Premise: The man drank heavily at the party.
Question: effect
  Choice1 - He had a headache the next day.
  Choice2 - He had a runny nose the next day.

Label: Choice1
_________________________

Premise: The bowling ball knocked over the bowling pins.
Question: cause
  Choice1 - The man rolled the bowling ball down the alley.
  Choice2 - The man dropped the bowling ball on his foot.

Label: Choice1
_________________________

Premise: The woman banished the children from her property.
Question: cause
  Choice1 - The children hit a ball into her yard.
  Choice2 - The children trampled through her garden.

Label: Choice2
_____